# API Nakala

## Packages

In [ ]:
using CSV
using DataFrames
using HTTP
using JSON
using Dates

## Identifiants

In [ ]:
path = @__DIR__
credentials = CSV.read(joinpath(path,"credentials", "credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "tnakala" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [ ]:
apitest = true

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else 
  apiurl = "https://api.nakala.fr"
end

## Création d'une collection

Une fois les identifiants chargés et l'API sélectionnée, entrez simplement un nom de collection pour la créer.

In [ ]:
collectionName = ""

url = joinpath(apiurl, "collections")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :status => "private",
  :metas =>  [Dict(
    :value => collectionName,
    :propertyUri => "http://nakala.fr/terms#title",
    :typeUri => "http://www.w3.org/2001/XMLSchema#string",
    :lang => "fr"
  )]
)

postCollection = HTTP.request("POST", url, headers, JSON.json(body)) # envoi des données pour la création de la collection
collectionResponse = JSON.parse(String(HTTP.payload(postCollection))) # réponse du server
collectionId = collectionResponse["payload"]["id"] # récupération de l'id de la collection
print("Identifiant collection : ", collectionId)

## Informations utilisateur

In [ ]:
url = joinpath(apiurl, "users", "me")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

userInfo = HTTP.request("GET", url, headers)
userInfoResponse = JSON.parse(String(HTTP.payload(userInfo))) # réponse du server
username = userInfoResponse["username"]
userGroupId = userInfoResponse["userGroupId"]
userInfoResponse

## Ressources utilisateur

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]

url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)
userDatas = HTTP.request("POST", url, headers)
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

## Supprimer des données d'une collection

In [ ]:
collectionIdentifier = ""
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

url = joinpath(apiurl, "collections", collectionIdentifier, "datas")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = datas

removeDatasFromCollection = HTTP.request("DELETE", url, headers, JSON.json(body)) # supprime les données listées plus haut de la collection
response = JSON.parse(String(HTTP.payload(removeDatasFromCollection))) # réponse du server

## Ajouter des données à une collection

In [ ]:
collectionIdentifier = "10.34847/nkl.5c519k06"
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

url = joinpath(apiurl, "collections", collectionIdentifier, "datas")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = datas

postDatasToCollection = HTTP.request("POST", url, headers, JSON.json(body)) # ajoute les données listées ci-dessus à une collection
response = JSON.parse(String(HTTP.payload(postDatasToCollection))) # réponse du server